In [1]:
%pip install openai

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 206.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-5d70a06f-9923-46ef-a3de-ec814123db94
    Can't uninstall 'typing_extensions'. No files were found to uninst

In [2]:
import requests
from base64 import b64encode
import json
import os
from openai import AzureOpenAI
from pyspark.sql.functions import *
from notebookutils.mssparkutils.credentials import getSecret
import ast

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 9, Finished, Available, Finished)

In [3]:
%run "/configuration"

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 11, Finished, Available, Finished)

In [4]:
keyvault_endpoint = "https://hackathonkv.vault.azure.net/"

# Get PAT Key
pat_key = getSecret(keyvault_endpoint, "azuredevoppat")

# Azure OpenAI
openai_endpoint = getSecret(keyvault_endpoint, "opeaiendpoint")
openai_key = getSecret(keyvault_endpoint, "openaikey")
openai_api_version = getSecret(keyvault_endpoint, "openaiapiversion")
openai_model = getSecret(keyvault_endpoint, "openaimodelname")

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 12, Finished, Available, Finished)

In [5]:
class AzureDevOps_API:
    def __init__(self, organization, token):
        self.organization = organization
        self.token = token

    def header_generate(self):
        authorization = str(b64encode(bytes(':'+ self.token, 'ascii')), 'ascii')
        headers = {'Accept': 'application/json','Authorization': 'Basic '+ authorization}
        return headers
    
    def get_wikipage_content(self, project, wikiIdentifier, includeContent=True, recursionLevel="OneLevel",api_version="7.2-preview.1"):
        if wikiIdentifier != None:
            url = f"https://dev.azure.com/{self.organization}/{project}/_apis/wiki/wikis/{wikiIdentifier}/pages?recursionLevel={recursionLevel}&includeContent={includeContent}&api-version={api_version}"
            headers = self.header_generate()
            response = requests.get(url, headers=headers)

            # Check for HTTP errors 
            if response.status_code != 200:
                print(f"Error: {response.status_code} - {response.text}")
                return None 
            else:
                return response.json()
        else:
            return "No Wiki documetation"

    def search_workitem_in_azure(self, search_text, projects=None, api_version = "7.2-preview.1", top_result = 0):
        url = f"https://almsearch.dev.azure.com/{self.organization}/_apis/search/workitemsearchresults?api-version={api_version}"
        headers = self.header_generate()

        # Create the request body with the search keyword
        body = {
            "searchText": search_text,
            "$top": top_result,
            "filters": {
                "System.TeamProject": projects if projects else [],
            },
            "includeFacets": True,
            "includeSnippet": True
        }

        # Send the POST request
        response = requests.post(url, headers=headers, json=body)

        # Check the response status
        if response.status_code == 200:
            return response.json()  # Return the results as JSON
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    
    def export_json_to_file(self, response_data, folder_path, file_path, max_char = 5000):

        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            # print(f"{folder_path} created successfully.")
        # else: 
            # print(f"{folder_path} already created.")

        # try:
            # Attempt to convert the response_data to JSON format if it's not already a dict
            #if isinstance(response_data, dict):
            #    json_data = response_data
            #else:
                # Try to parse the response_data into a JSON object
            #    json_data = json.loads(response_data)
            
        # Convert JSON data to string
        json_str = json.dumps(response_data, ensure_ascii=False, indent=4)
        # Check if the length exceeds max_characters
        # if len(json_str) > max_char:
        #    json_str = json_str[:max_char] # Truncate to max_character length

        # Write the JSON data to the specified file path
        with open(file_path, 'w', encoding='utf-8') as json_file:
            json_file.write(json_str)
        # print(f"JSON data has been exported successfully to {file_path}")

        # except json.JSONDecodeError:
        #     print("The response data is not in valid JSON format.")
        # except Exception as e:
        #     print(f"An error occurred while exporting to JSON: {e}")

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 13, Finished, Available, Finished)

In [6]:
class AzOpenAI:
    def __init__(self, endpoint, api_key, api_version, model_name):
        self.endpoint = endpoint
        self.api_key = api_key
        self.api_version = api_version
        self.model_name = model_name
    
    def get_search_keyword(self, user_question):

        client = AzureOpenAI(
            azure_endpoint = self.endpoint,
            api_key = self.api_key,
            api_version = self.api_version
        )

        response = client.chat.completions.create(
            model = self.model_name,
            response_format = {"type": "text"},
            messages=[
                {"role": "system", "content": """
                Define the most important topic keyword based on the user question below:
                If user write any phrases without spaces between the words, let treat them as one keyword. 
                If there are more one topic keyword, provide them as a list data type. 
                If there is just one topic keyword, provide it as a string
                """},
                {"role": "user", "content": user_question}
            ]
        )
        result = response.choices[0].message.content
        return result
    
    def get_answers(self, json_content, keyword):

        client = AzureOpenAI(
            azure_endpoint = self.endpoint,
            api_key = self.api_key,
            api_version = self.api_version
        )


        response = client.chat.completions.create(
            model = self.model_name,
            response_format = {"type": "text"},
            messages=[
                {"role": "system", "content": f"""You are an AI assistant that helps to find the information from text file, the keyword here is {keyword} and write the short comprehensive answer aboput that keyword as text."""},
                {"role": "user", "content": json_content}
            ]
        )
        result = response.choices[0].message.content
        return result

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 14, Finished, Available, Finished)

In [7]:
def get_json(path):
    if 'lakehouse/default' in path:
        path = path.split("lakehouse/default/")[1]
    json_dict = spark.read.option("multiline", "true").json(path).select('facets')
    json_rows = json_dict.collect()
    # Convert rows to a list of dictionaries
    json_dict_list = [row.asDict() for row in json_rows]
    return json_dict_list

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 15, Finished, Available, Finished)

In [14]:
def project_iq(user_question):
    openai = AzOpenAI(openai_endpoint, openai_key, openai_api_version, openai_model)
    keyword_result = openai.get_search_keyword(user_question)

    if "[" in keyword_result and "]" in keyword_result:
        keyword_result = ast.literal_eval(keyword_result)
        for keyword in keyword_result: 
            devops = AzureDevOps_API(configuration_para['organization'], pat_key)
            search_response = devops.search_workitem_in_azure(keyword, configuration_para['projects'])
            devops.export_json_to_file(search_response, folder_path, folder_path + f"{keyword.replace(' ', '')}" + ".json")
    else:
        devops_api = AzureDevOps_API(configuration_para['organization'], pat_key)
        search_response = devops_api.search_workitem_in_azure(keyword_result, configuration_para['projects'])
        devops_api.export_json_to_file(search_response, folder_path, folder_path + f"{keyword_result.replace(' ', '')}" + ".json")

    final_answer = "" 
    if isinstance(keyword_result, list):
        for index, keyword in enumerate(keyword_result):
            json_file = get_json(folder_path + f"{keyword.replace(' ', '')}" + ".json")
            openai = AzOpenAI(openai_endpoint, openai_key, openai_api_version, openai_model)
            answer = f"{index+1}. " + f"{openai.get_answers(str(json_file), keyword)}"
            final_answer += answer
    else:
        json_file = get_json(folder_path + f"{keyword_result.replace(' ', '')}" + ".json")
        openai = AzOpenAI(openai_endpoint, openai_key, openai_api_version, openai_model)
        answer = openai.get_answers(str(json_file), keyword_result)
        final_answer = answer 
    return print(final_answer)

StatementMeta(, 383bf70c-1243-43bf-a33e-fb9f96ddd4c5, 22, Finished, Available, Finished)